<a href="https://colab.research.google.com/github/jtjohnson565/CSE_620_Final_Project/blob/main/Louvain%26Leiden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install python-louvain
#!pip install networkx

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from networkx.algorithms import community
import random
import pandas as pd
import community as community_louvain
try:
    import pygraphviz
    from networkx.drawing.nx_agraph import graphviz_layout
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import graphviz_layout
    except ImportError:
        raise ImportError("This example needs Graphviz and either "
                          "PyGraphviz or pydot")

In [4]:
df_nodes = pd.read_csv('stack_network_nodes.csv')
df_edges = pd.read_csv('stack_network_links.csv')

In [ ]:
#df_nodes.head()

In [ ]:
#df_edges.head()

,source,target,value
0,azure,.net,20.933192
1,sql-server,.net,32.322524
2,asp.net,.net,48.407030
3,entity-framework,.net,24.370903
4,wpf,.net,32.350925


In [5]:
# get edges and weight
edges = df_edges[['source', 'target']].values.tolist()
weights = [float(l) for l in df_edges.value.values.tolist()]

In [6]:
# Make Graph and apply weight
G = nx.Graph(directed=True)
G.add_edges_from(edges)
for cnt, a in enumerate(G.edges(data=True)):
    G.edges[(a[0], a[1])]['weight'] = weights[cnt]

In [7]:
#!pip install python-louvain
#!pip install networkx

import community as community_louvain # or any other alias you prefer
def simple_Louvain(G):
    random.seed()
    """ Louvain method github basic example"""
    partition = community_louvain.best_partition(G)
    pos = graphviz_layout(G)

    # Calculate modularity
    modularity = community_louvain.modularity(partition, G)
    print(f"Louvain Modularity: {modularity}") # Print the modularity value

    max_k_w = []
    for com in set(partition.values()):
        list_nodes = [nodes for nodes in partition.keys()
                      if partition[nodes] == com]
        max_k_w = max_k_w + [list_nodes]

    node_mapping = {}
    map_v = 0
    for node in G.nodes():
        node_mapping[node] = map_v
        map_v += 1

    community_num_group = len(max_k_w)
    color_list_community = [[] for i in range(len(G.nodes()))]

    # color
    for i in G.nodes():
        for j in range(community_num_group):
            if i in max_k_w[j]:
                color_list_community[node_mapping[i]] = j

    return G, pos, color_list_community, community_num_group, max_k_w, modularity # Return modularity

In [9]:
!pip install python-igraph # Install necessary packages if not already installed
!pip install leidenalg
import leidenalg as la
import igraph

def simple_Leiden(G):
    """ Leiden method similar to Louvain example"""
    random.seed()
    # Convert the NetworkX graph to igraph using the igraph package
    G_igraph = igraph.Graph.from_networkx(G)

    # Apply the Leiden algorithm
    partition = la.find_partition(G_igraph, la.ModularityVertexPartition)

    # Get community structure
    communities = {}
    for i, community_id in enumerate(partition.membership):
        if community_id not in communities:
            communities[community_id] = []
        communities[community_id].append(list(G.nodes())[i])

    # Create color list for communities
    community_num_group = len(communities)
    color_list_community = [0] * len(G.nodes())
    for node, community_id in enumerate(partition.membership):
        color_list_community[node] = community_id

    # Assuming you have 'pos' from your Louvain code (or generate it)
    pos = nx.spring_layout(G)  # Or use your existing 'pos'

    # Calculate modularity using igraph's modularity function
    modularity = G_igraph.modularity(partition.membership)

    return G, pos, color_list_community, community_num_group, communities, modularity # Return modularity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00


In [10]:
G, pos, color_list_community, community_num_group, max_k_w, modularity = simple_Louvain(G)

Louvain Modularity: 0.756697283135632


In [18]:
G1, pos1, color_list_community1, community_num_group1, max_k_w1, modularity1 = simple_Leiden(G)
print(f"Leiden Modularity: {modularity}")  # Print the modularity

Leiden Modularity: 0.7307122032486463


In [15]:
#Visualize Louvain
edges = G.edges()
Feature_color_sub = color_list_community
node_size = 350

fig = plt.figure(figsize=(200, 100))
im = nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=Feature_color_sub, cmap='jet', vmin=0, vmax=community_num_group)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos, font_size=50, font_color="black")
plt.xticks([])
plt.yticks([])
plt.colorbar(im)
plt.show(block=False)

In [19]:
#Visualize Leiden

edges = G1.edges()
Feature_color_sub = color_list_community1
node_size = 70

fig = plt.figure(figsize=(200, 100))
im = nx.draw_networkx_nodes(G1, pos1, node_size=node_size, node_color=Feature_color_sub, cmap='jet', vmin=0, vmax=community_num_group1)
nx.draw_networkx_edges(G1, pos1)
nx.draw_networkx_labels(G1, pos1, font_size=50, font_color="black")
plt.xticks([])
plt.yticks([])
plt.colorbar(im)
plt.show(block=False)